In [2]:
import cirq
import sympy
from scipy.optimize import *
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Задача коммивояжёра

Ответом на задачу является последовательность, в которой мы посещаем города.

Первый город можно выбрать $n$ способами. Если граф полный, то второй город можно выбрать $n-1$ способом, третий $n-2$ способами и так далее, суммарно $n!$ возможных последовательностей городов.

Будем искать незамкнутые пути.

In [3]:
# n_towns = 4

In [4]:
def create_cost_matrix(n_towns):
   """Рандомная матрица с положительными элементами, 
   которая хранит длины рёбер между каждой парой городов.
   Диагональные элементы ничего не значат """

   return np.random.random((n_towns, n_towns))

In [53]:
# cost_matrix_4 = create_cost_matrix(4)

cost_matrix_4 = np.array([[np.nan    , 0.35271991, 0.96262685, 0.11727604],
                          [0.13505078, np.nan    , 0.63915344, 0.43149425],
                          [0.58432224, 0.83676812, np.nan    , 0.4879146 ],
                          [0.14998587, 0.45394107, 0.2140258 , np.nan    ]])

cost_matrix_4 = np.array([[0         , 0.35271991, 0.96262685, 0.11727604],
                          [0.13505078, 0         , 0.63915344, 0.43149425],
                          [0.58432224, 0.83676812, 0         , 0.4879146 ],
                          [0.14998587, 0.45394107, 0.2140258 , 0         ]])

In [7]:
def cost_of_permutation(cost_matrix, town_sequence):
  cost = 0.0

  for i in range(len(town_sequence)-1):
    cost += cost_matrix [town_sequence[i]] [town_sequence[i+1]]
  
  return cost

#### Аналитическое решение

Переберём все перестановки в лексикографическом порядке.

Будем итерироваться по номеру перестановки. Сначала переведём его в факториальную систему счисления [(factoradic)](https://en.wikipedia.org/wiki/Factorial_number_system). Полученные цифры будут совпадать с [кодом Лемера](https://en.wikipedia.org/wiki/Lehmer_code) перестановки. 

Затем по коду Лемера вычислим саму перестановку и затем её стоимость.

In [8]:
# Для перевода из десятичной системы счисления в факториальную 
# понадобятся факториалы, поэтому вычислим их заранее:
#
# factorials[n] == n!

factorials = [1]
for i in range(30):
  # Предполагаю, что n_towns < 30
  factorials.append(factorials[i] * (i+1))

In [9]:
def factoradic_from_perm_number(perm_number : int, factoradic_array):
  """Преобразует число в факториальную систему счисления.
  Записывает полученные цифры в factoradic_array.
  Нулевая цифра в массиве - самая значимая. Последняя цифра всегда равна 0.
  Алгоритм из https://en.wikipedia.org/wiki/Factorial_number_system#Definition """

  N = perm_number
  n_towns = len(factoradic_array)
  assert N < factorials[n_towns]

  for i in range(len(factoradic_array)):
    factoradic_array[i] = N // factorials[n_towns - i - 1]
    N %= factorials[n_towns - i - 1]
  
  return factoradic_array

In [10]:
def permutation_from_factoradic(nums : np.ndarray):
  """Перезаписывает массив nums.
  Если изначально там находилось число в факториальной системе, то потом там будет находиться перестановка.
  Города в полученной перестановке нумеруются с 0. 
  Алгоритм из https://en.wikipedia.org/wiki/Lehmer_code#Encoding_and_decoding """
  for i in range(len(nums)-2, -1, -1):
    for j in range(i+1, len(nums)):
      if(nums[j] >= nums[i]):
        nums[j] += 1
  return nums

In [11]:
# Тест из Википедии
permutation_from_factoradic([1, 4, 0, 3, 1, 1, 0]) \
                         == [1, 5, 0, 6, 3, 4, 2]

True

In [12]:
def analytical(cost_matrix):

  n_towns = len(cost_matrix)

  factoradic_array = np.ndarray(n_towns, int)

  answers_array = []

  min_cost = np.inf
  opt_perm_number = np.nan

  for perm_number in range(factorials[n_towns]):
    town_sequence = permutation_from_factoradic(
                        factoradic_from_perm_number(perm_number, factoradic_array)
                    )
    cost = cost_of_permutation(
        cost_matrix,
        town_sequence
        )

    answers_array.append([town_sequence.copy(), cost])

    if cost < min_cost:
      min_cost = cost
      opt_perm_number = perm_number

  # return {"min_cost": min_cost, 
  #         "order_of_towns": permutation_from_factoradic(
  #             factoradic_from_perm_number(opt_perm_number, factoradic_array)
  #           ) }
  return {"opt_perm_number": opt_perm_number,
          "answers_table": pd.DataFrame(answers_array, columns = ["order of towns", "cost"])}

In [13]:
anal = analytical(cost_matrix_4)
anal['answers_table'].loc[[anal['opt_perm_number']]]
# = Оптимальный путь:

,order of towns,cost
7,"[1, 0, 3, 2]",0.466353


In [14]:
anal['answers_table']

,order of towns,cost
0,"[0, 1, 2, 3]",1.479788
1,"[0, 1, 3, 2]",0.998240
2,"[0, 2, 1, 3]",2.230889
3,"[0, 2, 3, 1]",1.904483
4,"[0, 3, 1, 2]",1.210371
5,"[0, 3, 2, 1]",1.168070
6,"[1, 0, 2, 3]",1.585592
7,"[1, 0, 3, 2]",0.466353
8,"[1, 2, 0, 3]",1.340752
9,"[1, 2, 3, 0]",1.277054


In [15]:
px.scatter(anal["answers_table"]["cost"], template="plotly_dark", labels = {"index": "Номер перестановки", "value": "Cost"})

## Цепь

Пусть $m = \lceil \log_2 n! \rceil$ $\quad$ ($n$ -- число городов)

Будем в квантовом состоянии записывать номер перестановки в виде двоичного числа со знаком, для этого понадобится $m + 1 = O(n \log n)$ кубитов.

В начале установим sign qubit в состояние $|1\rangle$, а с оставшимися кубитами поделаем произвольные параметрические гейты. В результате получится некоторое отрицательное число $x \in [-2^m,\ 0)$

Затем сделаем операцию $x \mapsto x + n!$ по [алгоритму из Интернета,](https://cran.r-project.org/web/packages/qsimulatR/vignettes/addbyqft.pdf) использующему QFT. В результате получится число 

$$x \in [-2^m + n!,\ n!), \tag{1}$$

причём в силу определения числа $m$ выполнено $2^{m-1} \leqslant n! \leqslant 2^m$

А хотим мы получить валидный номер перестановки, т.е. $x \in [0,\ n!)$. Для этого нужно что-то сделать с отрицательными числами.

Я предлагаю, если число отрицательное, применить побитовое $NOT$, которое эквивалентно операции $x \mapsto -x - 1$.

Положительная часть промежутка $(1)$ останется неизменной, а отрицательная преобразуется так:
$$ [-2^m + n!,\ -1] \ \mapsto\ [0,\ 2^m - n! - 1]$$
Проверим, что мы не вышли за пределы промежутка $[0,\ n!)$. Действительно, из $2^{m-1} \leqslant m$ следует, что $2^m - n! - 1 < n!$

Таким образом, какие бы параметрические гейты мы не делали, на выходе получится валидный номер перестановки $x \in [0, n!)$. 

И наоборот, любой допустимый номер перестановки можно получить подходящими параметрическими гейтами.

Общая схема:

```
sign ───────────X─────────────┬──────┬────@┬───
     ──┬─────────────────┬────┤      ├─────┼X────M─┐
     ──┤ параметрические ├────┤ + n! ├─────┼X────M─┤ ← число от 0
     ──┤      гейты      ├────┤      ├─────┼X────M─┤   до n!
     ──┴─────────────────┴────┴──────┴─────┴X────M─┘
                           ↑            ↑
                число от -2^m          число от -2^m + n!
                         до 0          до n!
```

В конце схемы я применяю побитовое отрицание к отрицательным числам. Для этого к каждому кубиту применяю CNOT, используя sign qubit в качестве контролирующего кубита. Сам sign qubit после этого отбрасывается.

Возьмём в качестве параметрических гейтов пары из слоёв $R_x(a_i)$ и $CNOT$:

```
──┬─────────────────┬──            ──Rx(a_0)──@┬─────────────
──┤ параметрические ├──     ___    ──Rx(a_1)───┴X───@┬───────
──┤      гейты      ├──     ───    ──Rx(a_2)─────────┴X───@┬─ (повторяется n_layer_pairs раз с разными a_i)
──┴─────────────────┴───           ──Rx(a_3)───────────────┴X

```

In [16]:
class RxCirquit:

   def __init__(self, n_towns, n_layer_pairs):
      n_qubits = int(np.ceil(np.log2(factorials[n_towns]))) + 1

      self.qubits = cirq.LineQubit.range(n_qubits)
      self.circuit = cirq.Circuit()
      self.simulator = cirq.Simulator()

      self._town_sequence = np.ndarray(n_towns, int)

      self._angle_names = [f'a_{i}' for i in range(n_layer_pairs * (n_qubits-1))]
      angle_symbols = sympy.symbols(self._angle_names)

    # Параметрические гейты
      for layer_pair in range(n_layer_pairs):
          rx_gates = [cirq.rx(angle)(qubit)
                      for (angle, qubit)
                      in zip(
                        angle_symbols[layer_pair*(n_qubits-1) : (layer_pair+1)*(n_qubits-1)],
                        self.qubits[1:]) 
                    ]
          self.circuit.append(cirq.Moment(rx_gates))

          cnot_gates = [cirq.CNOT(q1, q2)
                        for (q1, q2)
                        in zip(self.qubits, self.qubits[2:]) ]
          self.circuit.append(cnot_gates)

    # Гейты для того, чтобы измеренный номер перестановки не превышал n_towns!
      self.circuit.append(cirq.X(self.qubits[0]))
      # sign qubit — делаем отрицательное число


      # Прибавим n_towns! к числу, которое меньше 0
      # Получим число, которое меньше n_towns!
      self.circuit.append(
          cirq.qft(*self.qubits, without_reverse = True)
      )

      for j in range(n_qubits):
        self.circuit.append(
            cirq.ZPowGate(
                exponent = 2*factorials[n_towns] / 2**(n_qubits - j)
              )(self.qubits[j])
        )


      self.circuit.append(
          cirq.qft(*self.qubits, inverse=True, without_reverse = True)
      )

      # Если получили отрицательное число, то при помощи побитового CNOT
      # сделаем отображение X -> -X-1
      # Можно показать, что номер не станет слишком большим
      for j in range(1, n_qubits):
        self.circuit.append(
            cirq.CNOT(self.qubits[0], self.qubits[j])
        )
      
      measurements = cirq.measure_each(*self.qubits[1:])
      self.circuit.append(cirq.Moment(measurements))


   def measure(self, angles, repetitions = 1):

      params = cirq.ParamResolver({
         name: value for (name, value) in
         zip(self._angle_names, angles)
      })

      return self.simulator.run_sweep(self.circuit, params, repetitions)[0]


   def cost_of_bits(self, bits, cost_matrix):
      "Вычисляет по последовательности битов перестановку и её стоимость"
      return cost_of_permutation(
          cost_matrix,
          permutation_from_factoradic(
              factoradic_from_perm_number(
                  cirq.big_endian_bits_to_int(bits),
                  self._town_sequence
              )
          )
      )


   def costs(self, angles, cost_matrix, repetitions):
      "Повторяет измерения и каждый раз вычисляет кост"

      measurementDataFrame = self.measure(angles, repetitions).data

      return measurementDataFrame.apply(
          self.cost_of_bits, 
          axis = 1, raw = True, args = (cost_matrix,)
          )


   def optimize_without_constraints(self, cost_matrix, 
                                    optimizer = differential_evolution,
                                    repetitions = 100,
                                    **optimizer_kwargs):
      """
      Вызывает функцию, заданную в аргументе optimizer.
      С её помощью подбирает параметры гейтов так,
      чтобы решить задачу коммивояжёра.
      Маршрут вычисляется на основе результатов измерений снова и снова,
      а затем суммарная длина пути усредняется. 
      Именно эту усреднённую длину пути мы пытаемся минимизировать.

      Args:

       cost_matrix - матрица размером n_towns × n_towns такая, что
                     cost_matrix[i][j] есть длина пути между городами i и j

       optimizer - одна из функций https://docs.scipy.org/doc/scipy/reference/optimize.html#global-optimization

       repetitions - число повторений для усреднения коста. 
                     Для градиентных методов нужна высокая точность, 
                     так как они считают разность между близкими величинами.
                     Могут потребоваться миллионы повторений.
      """

      cost_lambda = lambda angles : self.costs(angles, cost_matrix, repetitions).mean()

      if optimizer == basinhopping or optimizer == minimize:
         if 'x0' not in optimizer_kwargs:
            optimizer_kwargs['x0'] = [2 * np.pi * (np.random.random() - 0.5)
                                    for _ in self._angle_names]
      else:
         if 'bounds' not in optimizer_kwargs:
            optimizer_kwargs['bounds'] = [(-np.pi, np.pi) for _ in self._angle_names]

      return optimizer(cost_lambda, **optimizer_kwargs)


   def __str__(self):
      return self.circuit.__str__()

   def _repr_pretty_(self, *args):
      "Text output in Jupyter"
      return self.circuit._repr_pretty_(*args)

In [245]:
RxCircuit_4_2 = RxCirquit(n_towns = 4, n_layer_pairs = 2)
RxCircuit_4_2

                ┌──┐   ┌──┐             ┌──┐   ┌──┐
0: ──────────────@───────────────────────@──────X─────qft[norev]───Z^0.75───qft[norev]^-1───@───@───@───@───@───────
                 │                       │            │                     │               │   │   │   │   │
1: ───Rx(a_0)────┼@───────────Rx(a_5)────┼@───────────#2───────────S^-1─────#2──────────────X───┼───┼───┼───┼───M───
                 ││                      ││           │                     │                   │   │   │   │
2: ───Rx(a_1)────X┼─────@─────Rx(a_6)────X┼─────@─────#3───────────Z────────#3──────────────────X───┼───┼───┼───M───
                  │     │                 │     │     │                     │                       │   │   │
3: ───Rx(a_2)─────X─────┼@────Rx(a_7)─────X─────┼@────#4───────────Z^0──────#4──────────────────────X───┼───┼───M───
                        ││                      ││    │                     │                           │   │
4: ───Rx(a_3)───────────X┼────Rx(a_8)───

In [15]:
# Success, 1.5 min
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, 
                                            shgo, 
                                            repetitions = 100, 
                                            sampling_method = 'sobol', 
                )

     fun: 0.4663526200000001
    funl: array([0.46635262])
 message: 'Optimization terminated successfully.'
    nfev: 139
     nit: 2
   nlfev: 11
   nlhev: 0
   nljev: 1
 success: True
       x: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
      xl: array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [16]:
# Success, 3 min
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, differential_evolution)

     fun: 0.4663526200000001
 message: 'Optimization terminated successfully.'
    nfev: 32891
     nit: 216
 success: True
       x: array([-0.0917221 , -2.99309398,  3.05589482, -1.16475367, -0.50105521,
       -0.03506382, -0.07196514,  0.2176334 , -1.91275572, -2.74809019])

In [17]:
# Success, 2 min
# Можно попробовать с другой strategy
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, dual_annealing)

     fun: 0.4663526200000001
 message: ['Maximum number of iteration reached']
    nfev: 26447
    nhev: 0
     nit: 1000
    njev: 586
  status: 0
 success: True
       x: array([ 0.13195791,  3.13285266, -0.03904781,  0.73534826, -0.54798557,
       -0.05508556, -0.01845043, -3.10008193,  2.36314467,  0.77716622])

In [18]:
# Wrong Result, 15 sec
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, minimize, 10**4, method='cobyla')

     fun: 0.47010476591700007
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 126
  status: 1
 success: True
       x: array([-3.10535947,  3.1775052 ,  3.14658624,  4.10955368,  3.85471959,
        3.21259122,  3.16777168,  0.03775732, -0.90496329, -3.82700013])

#### Испытания

Классические оптимизаторы, которые каким-то чудом работают:
1. `scipy.optimize.differential_evolution`
2. `dual_annealing`
3. `shgo` с параметром `sampling_method='sobol'`

`COBYLA` не работает.

Все оптимизаторы из `scipy` в лучшем случае (`COBYLA` и `shgo`) используют nfev ≈ 150, что значительно больше n! = 24. Т. е. полный перебор работает быстрее.

Любопытно, что `differential_evolution` (вопреки названию) не использует дифференциирование.

Далее идут неудачные результаты тестов других оптимизаторов:

In [19]:
# Fail, 4 min 
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, minimize, repetitions = 10**4, method = 'nelder-mead')

 final_simplex: (array([[-1.30916924,  2.71295924, -0.32444764, -0.91288964, -2.62506555,
        -1.75219385,  1.82661744, -0.84445589, -0.67612808, -0.85639483],
       [-1.30916924,  2.71295924, -0.32444764, -0.91288964, -2.62506555,
        -1.75219385,  1.82661744, -0.84445589, -0.67612808, -0.85639483],
       [-1.30916924,  2.71295924, -0.32444764, -0.91288964, -2.62506555,
        -1.75219385,  1.82661744, -0.84445589, -0.67612808, -0.85639483],
       [-1.30916924,  2.71295924, -0.32444764, -0.91288964, -2.62506555,
        -1.75219385,  1.82661744, -0.84445589, -0.67612808, -0.85639483],
       [-1.30916924,  2.71295924, -0.32444764, -0.91288964, -2.62506555,
        -1.75219385,  1.82661744, -0.84445589, -0.67612808, -0.85639483],
       [-1.30916924,  2.71295924, -0.32444764, -0.91288964, -2.62506555,
        -1.75219385,  1.82661744, -0.84445589, -0.67612808, -0.85639483],
       [-1.30916924,  2.71295924, -0.32444764, -0.91288964, -2.62506555,
        -1.75219385,  1.8266

In [112]:
# Wrong or approximate answer, 2 min 
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, minimize, repetitions = 10**4, method = 'powell')

   direc: array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])
     fun: 0.4664227917340001
 message: 'Optimization terminated successfully.'
    nfev: 693
     nit: 4
  status: 0
 success: True
       x: array([-3.15685436e+00, -1.54750690e-03,  3.14268385e+00, -2.08925369e-03,
        2.01946626e+00, -3.13099056e+00,  3.14529293e+00,  3.13544571e+00,
       -2.47049953e-02, -2.02884605e+00])

In [21]:
# Fail, 4m sec 
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, minimize, repetitions = 10**4, method = 'CG')

     fun: 1.249236038818
     jac: array([1039149.14204092, 1317840.20429951,  380067.72514737,
        717307.16257289,  674113.54909356, 1240907.79634239,
        686435.35949557,  737966.37932757,  214615.3977128 ,
        854547.81874701])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 401
     nit: 3
    njev: 35
  status: 2
 success: False
       x: array([-1.21525608,  0.64685367,  0.60727529,  3.12168778,  1.42938628,
       -0.26930508, -2.00201487,  2.42059577,  1.87382361, -0.77242461])

In [22]:
# Fail, 5 min
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, minimize, 10**5, method='BFGS')

      fun: 1.3679762723647995
 hess_inv: array([[  1.57410899,   1.91168013,  -1.72073345,  -1.3138976 ,
          6.63634128,   0.74393882,   1.63136604,  -5.95791173,
         -6.01613926,  -4.81054263],
       [  1.91168013,   4.91700583,  -3.73630275,  -2.25680153,
         13.2932785 ,   1.19769991,   3.78344043, -11.44079575,
        -12.22585049,  -9.62561708],
       [ -1.72073345,  -3.73630275,   4.03302901,   1.94000347,
        -12.13431046,  -1.55250039,  -3.46579121,  10.34306821,
         10.77309056,   8.43899988],
       [ -1.3138976 ,  -2.25680153,   1.94000347,   2.02528768,
         -7.25015447,  -0.79444289,  -2.26275503,   6.00077962,
          6.52231046,   5.05644178],
       [  6.63634128,  13.2932785 , -12.13431046,  -7.25015447,
         45.3622782 ,   4.42613581,  12.83345786, -37.87779306,
        -40.4117499 , -31.7111538 ],
       [  0.74393882,   1.19769991,  -1.55250039,  -0.79444289,
          4.42613581,   1.03060953,   1.31164645,  -3.82401963,
      

In [23]:
# Fail, 30 sec
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, minimize, 10**4, method='L-BFGS-B')

      fun: 1.323145629146
 hess_inv: <10x10 LbfgsInvHessProduct with dtype=float64>
      jac: array([  204341.11497324,   230162.30959882,  -468061.04994808,
        -249968.11091916,  -208775.83046881,  -116783.04700975,
         121867.69084068, -1013832.93546151,  -606143.58681902,
        -238490.0352494 ])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 308
      nit: 3
     njev: 28
   status: 0
  success: True
        x: array([-0.58077443,  2.29720482,  0.51494906, -1.51338284,  3.10602678,
       -2.55284896,  2.70628379,  1.54238053,  0.48683189, -1.93392373])

In [24]:
# Fail, 30 sec
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, minimize, 10**4, method='TNC')

     fun: 1.356649087562
     jac: array([ 458110.3779981 ,  126595.85816939,  565908.25493929,
        538046.49536995, -562306.39217456, -229354.97799389,
        450996.72933739,  -91414.0169481 ,  714230.0187407 ,
        325136.05676628])
 message: 'Converged (|x_n-x_(n-1)| ~= 0)'
    nfev: 121
     nit: 2
  status: 2
 success: True
       x: array([-0.70018955, -1.14058802, -1.93371134, -1.31665688,  0.64010246,
       -2.11287222, -0.19075364,  0.10532458, -1.02892529, -0.53824389])

In [26]:
# Fail, 1.5 min
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, minimize, 10**4, method='SLSQP')

     fun: 1.2948185245780002
     jac: array([-360316.59540847, -124896.0825797 , -121321.07092074,
       -109091.87162349, -184002.75141861,   45563.36597985,
       -319895.75141086, -202094.0858786 , -246865.44405136,
       -222196.43999067])
 message: 'Optimization terminated successfully'
    nfev: 925
     nit: 44
    njev: 44
  status: 0
 success: True
       x: array([-71348.61276298,  21013.63474634, 112014.53673681,  56451.50396581,
       -19727.57848191,  32430.68800211, -38368.16237266,  26917.45229334,
        78579.71176846,  51717.38181607])

In [27]:
# Fail, 30 sec
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, minimize, 10**4, method='trust-constr')

         cg_niter: 48
     cg_stop_cond: 2
           constr: []
      constr_nfev: []
      constr_nhev: []
      constr_njev: []
   constr_penalty: 1.0
 constr_violation: 0
   execution_time: 37.73295211791992
              fun: 1.2207278997829998
             grad: array([ -48508.19359627,  127748.68982296,   78366.98805521,
       -218034.11473175, -205466.2325482 ,  -68369.58085357,
       -114627.93612298,  100192.99237136,  162431.82318858,
        -17274.85615304])
              jac: []
  lagrangian_grad: array([ -48508.19359627,  127748.68982296,   78366.98805521,
       -218034.11473175, -205466.2325482 ,  -68369.58085357,
       -114627.93612298,  100192.99237136,  162431.82318858,
        -17274.85615304])
          message: '`xtol` termination condition is satisfied.'
           method: 'equality_constrained_sqp'
             nfev: 385
             nhev: 0
              nit: 35
            niter: 35
             njev: 35
       optimality: 218034.114731752
           statu

In [28]:
# 7 min, fail
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, basinhopping, repetitions=1000)

                        fun: 1.1467525638800002
 lowest_optimization_result:       fun: 1.1467525638800002
 hess_inv: array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
      jac: array([2366463.66690934, 1418654.58436407,  610483.68591271,
       -476344.8846477 , 1616749.59465216,  527778.07297641,
       2607890.47004233, 2791511.82843214, 1188598.6024574 ,
        459251.42081045])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 265
      nit: 0
     njev: 23
   status: 2
  success: False
        x: array([ 4.55531833, -3.19294635,  0.35078097,  2.90749403,  0.25387454,
       -3.48732677, -0.07798095, -0.62492

In [116]:
# Fail, 2 min
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, 
                                                 shgo, 
                                                 repetitions = 1000, 
                                                 bounds = [(1, 1+2*np.pi) for _ in RxCircuit_4_2._angle_names])

     fun: 1.30595963304
    funl: array([1.30595963, 1.30607816, 1.30846972, 1.31134336, 1.31430259,
       1.3163793 , 1.31652658, 1.3181426 , 1.31914509, 1.32190803])
 message: 'Optimization terminated successfully.'
    nfev: 1156
     nit: 2
   nlfev: 131
   nlhev: 0
   nljev: 11
 success: True
       x: array([7.28318531, 1.        , 7.28318531, 7.28318531, 7.28318531,
       7.28318531, 1.        , 1.        , 7.28318531, 1.        ])
      xl: array([[7.28318531, 1.        , 7.28318531, 7.28318531, 7.28318531,
        7.28318531, 1.        , 1.        , 7.28318531, 1.        ],
       [1.        , 7.28318531, 7.28318531, 1.        , 1.        ,
        7.28318531, 7.28318531, 7.28318531, 1.        , 1.        ],
       [7.28318531, 1.        , 7.28318531, 7.28318531, 1.        ,
        1.        , 1.        , 7.28318531, 1.        , 7.28318531],
       [1.        , 7.28318531, 7.28318531, 7.28318531, 7.28318531,
        7.28318531, 1.        , 7.28318531, 7.28318531, 1.        

In [117]:
# 1 min, fail
RxCircuit_4_2.optimize_without_constraints(cost_matrix_4, 
                                                 shgo, 
                                                 repetitions = 10**4, 
                                                 sampling_method = 'halton', 
                    )

     fun: 1.197461658548
    funl: array([1.19746166])
 message: 'Optimization terminated successfully.'
    nfev: 111
     nit: 2
   nlfev: 11
   nlhev: 0
   nljev: 1
 success: True
       x: array([ 1.05756454,  2.74648521,  0.61104308,  1.33058515, -2.41377251,
        0.93495161, -0.51616518, -0.42167986, -0.63661757, -1.32514422])
      xl: array([[ 1.05756454,  2.74648521,  0.61104308,  1.33058515, -2.41377251,
         0.93495161, -0.51616518, -0.42167986, -0.63661757, -1.32514422]])

### Рассмотрим другую цепь (с Rz-гейтами)

In [118]:
class RxRzCircuit(RxCirquit):

  def __init__(self, n_towns, n_layer_triples):
    
      n_qubits = int(np.ceil(np.log2(factorials[n_towns]))) + 1

      self.qubits = cirq.LineQubit.range(n_qubits)
      self.circuit = cirq.Circuit()
      self.simulator = cirq.Simulator()

      self._town_sequence = np.ndarray(n_towns, int)

      self._angle_names = [f'a_{i}' for i in range(2 * n_layer_triples * (n_qubits-1))]
      angle_symbols = sympy.symbols(self._angle_names)

    # Параметрические гейты
      for layer_triple in range(n_layer_triples):
          rx_gates = [cirq.rx(angle)(qubit)
                      for (angle, qubit)
                      in zip(
                        angle_symbols[layer_triple*(n_qubits-1) : (layer_triple+1)*(n_qubits-1)],
                        self.qubits[1:]) 
                    ]
          self.circuit.append(cirq.Moment(rx_gates))

          rz_gates = [cirq.rz(angle)(qubit)
                      for (angle, qubit)
                      in zip(
                        angle_symbols[(layer_triple+1)*(n_qubits-1) : (layer_triple+2)*(n_qubits-1)],   
                        self.qubits[1:]) 
                    ]
          self.circuit.append(cirq.Moment(rz_gates))

          cnot_gates = [cirq.CNOT(q1, q2)
                        for (q1, q2)
                        in zip(self.qubits, self.qubits[2:]) ]
          self.circuit.append(cnot_gates)

    # Гейты для того, чтобы измеренный номер перестановки не превышал n_towns!
      self.circuit.append(cirq.X(self.qubits[0]))
      # sign qubit — делаем отрицательное число


      # Прибавим n_towns! к числу, которое меньше 0
      # Получим число, которое меньше n_towns!
      self.circuit.append(
          cirq.qft(*self.qubits, without_reverse = True)
      )

      for j in range(n_qubits):
        self.circuit.append(
            cirq.ZPowGate(
                exponent = 2*factorials[n_towns] / 2**(n_qubits - j)
              )(self.qubits[j])
        )


      self.circuit.append(
          cirq.qft(*self.qubits, inverse=True, without_reverse = True)
      )

      # Если получили отрицательное число, то при помощи побитового CNOT
      # сделаем отображение X -> -X-1
      # Можно показать, что номер не станет слишком большим
      for j in range(1, n_qubits):
        self.circuit.append(
            cirq.CNOT(self.qubits[0], self.qubits[j])
        )
      
      measurements = cirq.measure_each(*self.qubits[1:])
      self.circuit.append(cirq.Moment(measurements))


## Вывод

Из-за того, что нужно много раз повторять измерения (когда суперпозиция разных номеров перестановок), всё бессмысленно

In [119]:
RxRzCircuit_4_1 = RxRzCircuit(n_towns = 4, n_layer_triples = 1)
RxRzCircuit_4_1

                          ┌──┐   ┌──┐
0: ────────────────────────@──────X─────qft[norev]───Z^0.75───qft[norev]^-1───@───@───@───@───@───────
                           │            │                     │               │   │   │   │   │
1: ───Rx(a_0)───Rz(a_5)────┼@───────────#2───────────S^-1─────#2──────────────X───┼───┼───┼───┼───M───
                           ││           │                     │                   │   │   │   │
2: ───Rx(a_1)───Rz(a_6)────X┼─────@─────#3───────────Z────────#3──────────────────X───┼───┼───┼───M───
                            │     │     │                     │                       │   │   │
3: ───Rx(a_2)───Rz(a_7)─────X─────┼@────#4───────────Z^0──────#4──────────────────────X───┼───┼───M───
                                  ││    │                     │                           │   │
4: ───Rx(a_3)───Rz(a_8)───────────X┼────#5───────────Z^0──────#5──────────────────────────X───┼───M───
                                   │    │                     │

In [120]:
# Success, 1.5 min
RxRzCircuit_4_1.optimize_without_constraints(cost_matrix_4, 
                                            shgo, 
                                            repetitions = 100, 
                                            sampling_method = 'sobol', 
                )

     fun: 0.4663526200000001
    funl: array([0.46635262])
 message: 'Optimization terminated successfully.'
    nfev: 139
     nit: 2
   nlfev: 11
   nlhev: 0
   nljev: 1
 success: True
       x: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
      xl: array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [121]:
# Success, 3 min
RxRzCircuit_4_1.optimize_without_constraints(cost_matrix_4, differential_evolution)

     fun: 0.4663526200000001
 message: 'Optimization terminated successfully.'
    nfev: 9509
     nit: 61
 success: True
       x: array([ 0.18087251, -3.11266802, -3.06295076, -0.27645024,  0.1449418 ,
       -1.23763742,  2.31237694, -2.61008506,  0.37129562,  0.5738621 ])

In [122]:
# Success, 2 min
# Можно попробовать с другой strategy
RxRzCircuit_4_1.optimize_without_constraints(cost_matrix_4, dual_annealing)

     fun: 0.4663526200000001
 message: ['Maximum number of iteration reached']
    nfev: 24929
    nhev: 0
     nit: 1000
    njev: 448
  status: 0
 success: True
       x: array([-0.05610648, -3.05702992, -2.89435958,  0.22744969, -0.15671747,
        3.04693788, -2.78356467, -2.65554399, -0.4688722 , -1.94550051])

In [123]:
# Approximate Result, 15 sec
RxRzCircuit_4_1.optimize_without_constraints(cost_matrix_4, minimize, 10**4, method='cobyla')

     fun: 0.4674048509360001
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 140
  status: 1
 success: True
       x: array([-0.05234763, -3.12898678,  3.17406498, -6.31169016,  6.30585563,
       -4.22040843,  2.13472686, -3.46525386,  0.94004052, -1.47451736])

In [126]:
# Approximate Result, 15 sec
RxRzCircuit_4_1.optimize_without_constraints(cost_matrix_4, minimize, 10**4, method='cobyla')

     fun: 0.4677256329660001
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 131
  status: 1
 success: True
       x: array([ 0.02985708, -0.04655431,  0.0130951 , -0.04067813, -0.00664194,
        0.59273908, -2.45172399,  1.17981674,  0.42846024,  1.27041772])

In [127]:
# Approximate Result, 15 sec
RxRzCircuit_4_1.optimize_without_constraints(cost_matrix_4, minimize, 10**4, method='cobyla')

     fun: 0.4689735281690001
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 127
  status: 1
 success: True
       x: array([ 0.09964644, -0.00904898,  0.02179876,  0.01224183,  0.02763523,
        2.41652386,  0.67505693, -0.95754934, -1.1378796 , -0.34803205])

`COBYLA` начала давать стабильные почти точные результаты

In [124]:
# Fail, 4 min 
RxRzCircuit_4_1.optimize_without_constraints(cost_matrix_4, minimize, repetitions = 10**4, method = 'nelder-mead')

 final_simplex: (array([[-3.17509111, -2.24311423, -0.57652344, -0.02105428,  1.14193824,
        -2.71891431,  0.69892156, -2.44498138, -0.77860107, -2.23190932],
       [-3.17509111, -2.24311423, -0.57652344, -0.02105428,  1.14193824,
        -2.71891431,  0.69892156, -2.44498138, -0.77860107, -2.23190932],
       [-3.17509111, -2.24311423, -0.57652344, -0.02105428,  1.14193824,
        -2.71891431,  0.69892156, -2.44498138, -0.77860107, -2.23190932],
       [-3.17509111, -2.24311423, -0.57652344, -0.02105428,  1.14193824,
        -2.71891431,  0.69892156, -2.44498138, -0.77860107, -2.23190932],
       [-3.17509111, -2.24311423, -0.57652344, -0.02105428,  1.14193824,
        -2.71891431,  0.69892156, -2.44498138, -0.77860107, -2.23190932],
       [-3.17509111, -2.24311423, -0.57652344, -0.02105428,  1.14193824,
        -2.71891431,  0.69892156, -2.44498138, -0.77860107, -2.23190932],
       [-3.17509111, -2.24311423, -0.57652344, -0.02105428,  1.14193824,
        -2.71891431,  0.6989

In [125]:
# Wrong or approximate answer, 2 min 
RxRzCircuit_4_1.optimize_without_constraints(cost_matrix_4, minimize, repetitions = 10**4, method = 'powell')

   direc: array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])
     fun: 1.0890949400000003
 message: 'Optimization terminated successfully.'
    nfev: 450
     nit: 3
  status: 0
 success: True
       x: array([-3.14305514, -0.00755531, -0.00816975, -3.14585825,  3.12706186,
       -0.81370899,  4.30484461, -3.36446685,  2.73697235,  6.66645535])

## QAOA

Пусть в i-том регистре хранится город, в который идет в цепочке после i-того города.
Добавим нулевой город, который начинает и заканчивает маршрут.

In [ ]:
# np.roll()

In [23]:
# def encoding_to_permutation(enc):
#     perm = np.ndarray(len(enc) - 1, int)
#     perm[0] = enc[0]
#     next_town = perm[0]
#     for i in range(len(perm) - 1):
#         perm[i+1] = enc[perm[i]]
#     return perm - 1 # -1 для совместимости с 0-indexed arrays из предыдущего encoding

In [24]:
# encoding_to_permutation([2,3,0,1])

array([ 1, -1,  1])

In [431]:
class SwapTwoRegistries(cirq.Gate):
    def __init__(self, register_size, exponent):
        super(SwapTwoRegistries, self)
        self.register_size = register_size
        self.exponent = exponent
    
    def _num_qubits_(self):
        return 2 * self.register_size

    def _decompose_(self, qubits):
        first_register  = qubits[:self.register_size]
        second_register = qubits[self.register_size:]
        for (q1, q2) in zip(first_register, second_register):
            yield cirq.SwapPowGate(exponent = self.exponent)(q1, q2)

    def _circuit_diagram_info_(self, args):
        return cirq.CircuitDiagramInfo(["↓"] * self.register_size + ["↑"] * self.register_size, 
                                    exponent = self.exponent, exponent_qubit_index = 0)
    
    def _is_parametrized_(self):
        return isinstance(self.exponent, sympy.Symbol)

    def _resolve_parameters_(self, paramresolver, recursive):
        if self._is_parametrized_():
            self.exponent = paramresolver.param_dict[self.exponent.name]
        return self

In [439]:
class SwapAll(cirq.Gate):
    def __init__(self, n_registries, register_size, exponent):
        super(SwapAll, self)
        self.n_registries = n_registries
        self.register_size = register_size
        self.exponent = exponent

    def _num_qubits_(self):
        return self.n_registries * self.register_size

    def _decompose_(self, qubits):
        for n1 in range(self.n_registries):
            for n2 in range(n1 + 1, self.n_registries):
                yield SwapTwoRegistries(self.register_size, self.exponent)(
                    *qubits[n1 * self.register_size : (n1 + 1) * self.register_size], 
                    *qubits[n2 * self.register_size : (n2 + 1) * self.register_size]
                )

    def _circuit_diagram_info_(self, args):
        return cirq.CircuitDiagramInfo(["↑↓"] * self._num_qubits_(), exponent = self.exponent, exponent_qubit_index = 0)

    def _is_parametrized_(self):
        return isinstance(self.exponent, sympy.Symbol)

    def _resolve_parameters_(self, paramresolver, recursive):
        if self._is_parametrized_():
            return SwapAll(self.n_registries, self.register_size, paramresolver.param_dict[self.exponent.name])
        return self

In [473]:
class ExpOf_iFractionOfCost(cirq.Gate):
    """e^(i t cost[i][j])"""

    def __init__(self, cost_matrix, t):
        super(ExpOf_iFractionOfCost, self)
        self.t = t
        n_towns = len(cost_matrix)
        self.log_n_towns = int(np.ceil(np.log2(n_towns)))

        self.diagonal_before_exp = np.pad(cost_matrix, 
                                          ((0, 0), (0, 2**self.log_n_towns - n_towns)),
                                          constant_values = 0.0
                                     ).flatten()

    def _num_qubits_(self):
        return 2 * self.log_n_towns

    def _unitary_(self):
        return np.diag(np.exp(1j * self.t * self.diagonal_before_exp))

    def _circuit_diagram_info_(self, args):
        return [f"u({self.t})"] * self._num_qubits_()

    def _resolve_parameters_(self, paramresolver, recursive):
        self.t = paramresolver.param_dict[self.t.name]

In [474]:
class ExpOf_iCost(cirq.Gate):
    def __init__(self, cost_matrix, t):
        super(ExpOf_iCost, self)
        self.n_towns = len(cost_matrix)
        self.log_n_towns = int(np.ceil(np.log2(self.n_towns)))
        self.cost_matrix = cost_matrix
        self.t = t

    def _num_qubits_(self):
        return self.n_towns * self.log_n_towns

    def _decompose_(self, qubits):
        for n in range(self.n_towns - 1):
            yield ExpOf_iFractionOfCost(self.cost_matrix, self.t)(
                *qubits[n * self.log_n_towns : (n + 2) * self.log_n_towns]
            )

    def _circuit_diagram_info_(self, args):
        return [f"U({self.t})"] * self._num_qubits_()

    def _resolve_parameters_(self, paramresolver, recursive = True):
        self.t = paramresolver.param_dict[self.t.name]
        return self

In [475]:
class QaoaCircuit(RxCirquit):

  def __init__(self, n_towns, depth, cost_matrix):

      log_n_towns = int(np.ceil(np.log2(n_towns)))

      self.circuit = cirq.Circuit()
      self.simulator = cirq.Simulator()

      self._town_sequence = np.ndarray(n_towns, int)

    # Инициализация единичной перестановкой
      for n in range(n_towns):
        for (i, bit) in zip(range(log_n_towns), cirq.big_endian_int_to_bits(n, bit_count = log_n_towns)):
          if bit:
            self.circuit.append(cirq.X(cirq.GridQubit(n, i)))
            

    # # Перестановка, близкая к полной
      self.circuit.append(SwapAll(n_towns, log_n_towns, 1/2)(
        *cirq.GridQubit.rect(n_towns, log_n_towns)
      ))

    # Параметрические гейты
      self._mixing_powers = [f'm{i}' for i in range(depth)]
      mixing_powers = sympy.symbols(self._mixing_powers)

      self._hamilt_powers = [f'h{i}' for i in range(depth)]
      hamilt_powers = sympy.symbols(self._hamilt_powers)

      for p in range(depth):

        self.circuit.append(ExpOf_iCost(cost_matrix, hamilt_powers[p])(
          *cirq.GridQubit.rect(n_towns, log_n_towns)
        ))

        self.circuit.append(SwapAll(n_towns, log_n_towns, mixing_powers[p])(
          *cirq.GridQubit.rect(n_towns, log_n_towns)
        ))

      measurements = cirq.measure_each(*cirq.GridQubit.rect(n_towns, log_n_towns))
      self.circuit.append(cirq.Moment(measurements))


  def measure(self, angles, repetitions = 1):

      params = cirq.ParamResolver({
         name: value for (name, value) in
         zip(self._hamilt_powers + self._mixing_powers, angles)
      })

      return self.simulator.run_sweep(self.circuit, params, repetitions)[0]


  def cost_of_bits(self, bits, cost_matrix):
      "Вычисляет по последовательности битов перестановку и её стоимость"
      return cost_of_permutation(
          cost_matrix,
          permutation_from_factoradic(
              factoradic_from_perm_number(
                  cirq.big_endian_bits_to_int(bits),
                  self._town_sequence
              )
          )
      )

In [476]:
qaoaCircuit_4_1 = QaoaCircuit(4, 1, cost_matrix_4)
qaoaCircuit_4_1

(0, 0): ───────↑↓^0.5───U(h0)───↑↓^m0───M───
               │        │       │
(0, 1): ───────↑↓───────U(h0)───↑↓──────M───
               │        │       │
(1, 0): ───────↑↓───────U(h0)───↑↓──────M───
               │        │       │
(1, 1): ───X───↑↓───────U(h0)───↑↓──────M───
               │        │       │
(2, 0): ───X───↑↓───────U(h0)───↑↓──────M───
               │        │       │
(2, 1): ───────↑↓───────U(h0)───↑↓──────M───
               │        │       │
(3, 0): ───X───↑↓───────U(h0)───↑↓──────M───
               │        │       │
(3, 1): ───X───↑↓───────U(h0)───↑↓──────M───

In [477]:
qaoaCircuit_4_1.circuit.all_operations

<bound method AbstractCircuit.all_operations of cirq.Circuit([
    cirq.Moment(
        cirq.X(cirq.GridQubit(1, 1)),
        cirq.X(cirq.GridQubit(2, 0)),
        cirq.X(cirq.GridQubit(3, 0)),
        cirq.X(cirq.GridQubit(3, 1)),
    ),
    cirq.Moment(
        <__main__.SwapAll object at 0x17e404880>.on(cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(3, 0), cirq.GridQubit(3, 1)),
    ),
    cirq.Moment(
        <__main__.ExpOf_iCost object at 0x17e41c490>.on(cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(3, 0), cirq.GridQubit(3, 1)),
    ),
    cirq.Moment(
        <__main__.SwapAll object at 0x17e41ca60>.on(cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(3, 0), cirq.GridQubit(3, 1)),
    )

In [489]:
qaoaCircuit_4_1.measure(np.ones(6))

AttributeError: 'numpy.float64' object has no attribute 'name'

In [479]:
qaoaCircuit_4_1. cirq.ParamResolver({sympy.symbols("m0") : 1, sympy.symbols("h0") : 1})

AttributeError: 'QaoaCircuit' object has no attribute 'cirq'

In [480]:
cirq.resolve_parameters(RxCircuit_4_2, cirq.ParamResolver({sympy.symbols(f"a_{i}") : 1 for i in range(10)}))

                ┌──┐   ┌──┐             ┌──┐   ┌──┐
0: ──────────────@───────────────────────@──────X─────qft[norev]───Z^0.75───qft[norev]^-1───@───@───@───@───@───────
                 │                       │            │                     │               │   │   │   │   │
1: ───Rx(a_0)────┼@───────────Rx(a_5)────┼@───────────#2───────────S^-1─────#2──────────────X───┼───┼───┼───┼───M───
                 ││                      ││           │                     │                   │   │   │   │
2: ───Rx(a_1)────X┼─────@─────Rx(a_6)────X┼─────@─────#3───────────Z────────#3──────────────────X───┼───┼───┼───M───
                  │     │                 │     │     │                     │                       │   │   │
3: ───Rx(a_2)─────X─────┼@────Rx(a_7)─────X─────┼@────#4───────────Z^0──────#4──────────────────────X───┼───┼───M───
                        ││                      ││    │                     │                           │   │
4: ───Rx(a_3)───────────X┼────Rx(a_8)───

In [481]:
"""Define a custom gate with a parameter."""
class RotationGate(cirq.Gate):
    def __init__(self, theta):
        super(RotationGate, self)
        self.theta = theta

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        return np.array([
            [np.cos(self.theta), np.sin(self.theta)],
            [np.sin(self.theta), -np.cos(self.theta)]
        ]) / np.sqrt(2)

    def _circuit_diagram_info_(self, args):
        return f"R({self.theta})"

    # def _parameter_names_(self):
    #     if isinstance(self.theta, sympy.Symbol):
    #         return self.theta.name
    #     else:
    #         return NotImplemented
    
    # def _is_parametrized_(self, args):
    #     return isinstance(self.theta, sympy.Symbol)

    def _resolve_parameters_(self, paramresolver : cirq.ParamResolver, recursive):
        self.theta = paramresolver.param_dict[self.theta.name]
        return self

In [482]:
"""Use the custom gate in a circuit."""
circ = cirq.Circuit(
    RotationGate(sympy.symbols("theta2"))(cirq.LineQubit(0)),
    #cirq.Rx(rads = sympy.symbols("theta"))(cirq.LineQubit(0)),
    cirq.measure(cirq.LineQubit(0))
)

circ

0: ───R(theta2)───M───

In [483]:
cirq.Simulator().run_sweep(circ, cirq.ParamResolver({"theta2": np.pi}), repetitions=100)

[0=0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000]

In [484]:
ci = cirq.Circuit(
    ExpOf_iFractionOfCost(cost_matrix_4, 0.5)(*cirq.GridQubit.rect(2, 2)),
    cirq.measure_each(*cirq.GridQubit.rect(4, 2))
    )
ci

(0, 0): ───u(0.5)───M───
           │
(0, 1): ───u(0.5)───M───
           │
(1, 0): ───u(0.5)───M───
           │
(1, 1): ───u(0.5)───M───

(2, 0): ───M────────────

(2, 1): ───M────────────

(3, 0): ───M────────────

(3, 1): ───M────────────

In [488]:
cirq.Simulator().run(ci)

(0, 0)=0
(0, 1)=0
(1, 0)=0
(1, 1)=0
(2, 0)=0
(2, 1)=0
(3, 0)=0
(3, 1)=0

In [486]:
len((2,2,2,2,2,2,2,2))

8

In [459]:
ExpOf_iFractionOfCost(cost_matrix_4, 1)._unitary_().shape

(16,)